In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
import warnings
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model as loadModel
from sklearn.metrics import classification_report,confusion_matrix
import joblib

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = ''

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

In [ ]:
myDataDir = '../DATA/cell_images'
# CONFIRM THAT THIS REPORTS BACK 'test', and 'train'
os.listdir(myDataDir) 

In [ ]:
testPath = myDataDir+'/test/'
trainPath = myDataDir+'/train/'

In [ ]:
os.listdir(testPath)

In [ ]:
os.listdir(trainPath)

In [ ]:
paraCell = trainPath+'parasitized/'+os.listdir(trainPath+'parasitized')[0]
paraimg = imread(paraCell)
plt.imshow(paraimg)

In [ ]:
paraimg.shape

In [ ]:
uninfectedCell = trainPath+'uninfected/'+os.listdir(trainPath+'uninfected')[0]
uninfectedimg = imread(uninfectedCell)
plt.imshow(uninfectedimg)

In [ ]:
uninfectedimg.shape

In [ ]:
len(os.listdir(trainPath+'/parasitized'))

In [ ]:
len(os.listdir(trainPath+'/uninfected'))

In [ ]:
len(os.listdir(testPath+'/parasitized'))

In [ ]:
len(os.listdir(testPath+'/uninfected'))

# Find average dimensions of images

In [ ]:
dim1 = []
dim2 = []
count = 1
for imageFilename in tqdm(os.listdir(testPath+'uninfected')):
    img = imread(testPath+'uninfected'+'/'+imageFilename)
    d1,d2,colors = img.shape
    dim1.append(d1)
    dim2.append(d2)
    count += 1

In [ ]:
sns.jointplot(dim1,dim2)

In [ ]:
np.mean(dim1)

In [ ]:
np.mean(dim2)

In [ ]:
imageShape = (int(np.mean(dim1)),int(np.mean(dim2)),3)
imageShape

# Prepare the data

In [ ]:
imageGen = ImageDataGenerator(rotation_range=20, # rotate the image 20 degrees
                               width_shift_range=0.10, # Shift the pic width by a max of 5%
                               height_shift_range=0.10, # Shift the pic height by a max of 5%
                               # rescale=1/255, # Rescale the image by normalzing it.
                               shear_range=0.1, # Shear means cutting away part of the image (max 10%)
                               zoom_range=0.1, # Zoom in by 10% max
                               horizontal_flip=True, # Allow horizontal flipping
                               fill_mode='nearest', # Fill in missing pixels with the nearest filled value
                               dtype = 'float16'
                              )

In [ ]:
plt.imshow(paraimg)

In [ ]:
plt.imshow(imageGen.random_transform(paraimg))

In [ ]:
plt.imshow(imageGen.random_transform(paraimg))

# Create model

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=imageShape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=imageShape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=imageShape, activation='relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(128, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

earlyStop = EarlyStopping(monitor='val_loss',patience=2)

# Train model

In [ ]:
batchSize = 8

In [ ]:
target_size = imageShape[:2]

In [ ]:
#If you want to train on different image size, saved to a pickle file, uncomment below

'''target_size = joblib.load("target_size.pkl")
target_size'''

In [ ]:
trainimageGen = imageGen.flow_from_directory(directory = trainPath,
                                               target_size = target_size,
                                                color_mode='rgb',
                                               batch_size=batchSize,
                                               class_mode='binary', shuffle=True)

In [ ]:
testimageGen = imageGen.flow_from_directory(directory = testPath,
                                               target_size=target_size,
                                               color_mode='rgb',
                                               batch_size=batchSize,
                                               class_mode='binary', shuffle=False)

In [ ]:
trainimageGen.class_indices

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
model.summary()

In [ ]:
results = model.fit_generator(trainimageGen,epochs=100,
                              validation_data=testimageGen,
                              callbacks=[earlyStop])

# Evaluate model

In [ ]:
#If you want to load pre-trained network
    # 1) comment out the three 'losses' cells below this one
    # 2) uncomment  this cell below  

#model = loadModel('malaria_detectorNQM.h5')

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses[['loss','val_loss']].plot()

In [ ]:
losses[['accuracy','val_accuracy']].plot()

In [ ]:
model.metrics_names

In [ ]:
model.evaluate_generator(testimageGen)

In [ ]:
predProbabilities = model.predict_generator(testimageGen)

In [ ]:
predProbabilities

In [ ]:
testimageGen.classes

In [ ]:
predictions = predProbabilities > 0.5
predictions

In [ ]:
print(classification_report(testimageGen.classes,predictions))

In [ ]:
confusion_matrix(testimageGen.classes,predictions)

# sample prediction

In [ ]:
paraCell

In [ ]:
myimage = image.load_img(paraCell,target_size=target_size)
myimage

In [ ]:
type(myimage)

In [ ]:
myimage = image.img_to_array(myimage)

In [ ]:
type(myimage)

In [ ]:
myimage.shape

In [ ]:
myimage = np.expand_dims(myimage, axis=0)
myimage.shape

In [ ]:
model.predict(myimage)

In [ ]:
model.predict_classes(myimage)

In [ ]:
trainimageGen.class_indices

In [ ]:
testimageGen.class_indices

# Save model

In [ ]:
joblib.dump(imageShape[:2], "target_size.pkl")
model.save('malaria_detectorNQM.h5')

# Deployment Practice

In [ ]:
def return_prediction(model, target_size, sample_json):
    imgLoc = sample_json["imgLoc"]
    
    imgData = np.expand_dims(image.img_to_array(image.load_img(imgLoc, target_size = target_size)), axis = 0)
    
    classes = np.array(['parasitized', 'uninfected'])
    
    class_ind = model.predict_classes(imgData)
    
    return classes[class_ind][0][0]

In [ ]:
imgLoc = paraCell
cellExample = {'imgLoc' : imgLoc}
return_prediction(model, target_size, cellExample)

In [ ]:
imgLoc = uninfectedCell
cellExample = {'imgLoc' : imgLoc}
return_prediction(model, target_size, cellExample)